# Ceneo Scraper

## Załadowanie biblioteki

In [28]:
import requests
from bs4 import BeautifulSoup
import json 
import os

## Wysyłanie Zapytania do serwera

In [29]:
product_id = "138331381"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews_scroll"
response = requests.get(url)

## Pobieranie opini z kodu HTML strony

In [30]:
page_dom = BeautifulSoup(response.text, "html.parser")
print(type(page_dom))
opinions = page_dom.select("div.js_product-review")
print(type(opinions))

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.ResultSet'>


In [31]:
print(opinion)

<div class="user-post user-post__card js_product-review" data-entry-id="18371893">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/5.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
z...k</span>
<span class="user-post__author-recomendation">
<em class="recommended">Polecam</em>
</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 100.00%;"></span>
</span>
<span class="user-post__score-count">5/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2024-02-06 11:43:49">3 miesiące temu, </time>
<time datetime="2024-01-21 21:30:29">po 3 tygodniach</time> użytkowania            </span>
</span>
<div class="user-post__text">Ok dobre wykonanie</div>
<

## Ekstrakcja składowych pojedyńczej opinii
|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|[data-entry-id] |opinion_id|
|autor|span.user-post_author-name|author|
|rekomendacje|span.user-post__author-recomendation|recommendation|
|gwiazdki|span.user-post__score-count|star|
|treść|div.user-post__text|content|
|lista zalet|div.review-feature_title--positives ~ div.review-feature_item|pros|
|lista wad|div.review-feature_title--negatives ~ div.review-feature_item|cons|
|dla ilu przydatna|button.vote-yes > span|helpful|
|dla ilu nieprzydatna|button.vote-no > span|unhelpful|
|data wystawienia|span.user-post_published > time:nth-child(1)["datetime"]|publish_date|
|data zakupu|span.user-post_published > time:nth-child(2)["datetime"]|purchase_date|

In [32]:
opinion_id = opinion["data-entry-id"]
author = opinion.select_one("span.user-post__author-name").text
recommendation = opinion.select_one("span.user-post__author-recomendation").text
star = opinion.select_one("span.user-post__score-count").text
content = opinion.select_one("div.user-post__text").text
pros = opinion.select("div.review-feature__title--positives ~ div.review-feature__item")
cons = opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")
helpful = opinion.select_one("button.vote-yes > span").text
unhelpful = opinion.select_one("button.vote-no > span").text
publish_date = opinion.select_one('span.user-post__published > time:nth-child(1)').text
purchase_date = opinion.select_one('span.user-post__published > time:nth-child(2)').text

## Pobranie wszystkich opinii 

In [33]:
all_opinions = []
while(url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            "opinion_id": opinion["data-entry-id"],
            "author": opinion.select_one("span.user-post__author-name").text.strip(),
            "recommendation": opinion.select_one("span.user-post__author-recomendation").text.strip(),
            "star": opinion.select_one("span.user-post__score-count").text.strip(),
            "content": opinion.select_one("div.user-post__text").text.strip(),
            "pros": [p.text.strip() for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
            "cons": [c.text.strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
            "helpful": opinion.select_one("button.vote-yes > span").text.strip(),
            "unhelpful": opinion.select_one("button.vote-no > span").text.strip(),
            "publish_date": opinion.select_one('span.user-post__published > time:nth-child(1)').text.strip(),
            "purchase_date": opinion.select_one('span.user-post__published > time:nth-child(2)').text.strip(),
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl" + page_dom.select_one("a.pagination_next")["href"].strip()
    except TypeError: 
        url = None

None


## Zapis opinii o produkcie w JSON


In [34]:
if not os.path.exists("opinions"):
    os.makedirs("opinions")
with open(f"opinions/{product_id}.json","w",encoding="utf-8") as jf:
    json.dump(all_opinions, jf,indent = 4, ensure_ascii = False)